In [1]:
import pandas as pd
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import numpy as np
import numpy.random as rd

In [2]:
P_CORRECT = 0.8 #probability that an oracle predicts correctly
P_INCORRECT = 0.2 # probability that an oracle predicts incorrectly
DEFAULT_BITS = 8 #number of bits for binary representation for hamming dist calculation
STRONG = 1.2 #20% gain / decrease 
NORMAL = 1.02 #2% gain / decrease
MAX_INVESTMENT = 100 # Investment in case of All oracle say UP/DOWN

In [3]:
# compute the hamming distance between s1 and s2 which must binary strings of the same length
def hamming2(s1, s2):
    """Calculate the Hamming distance between two bit strings"""
    assert len(s1) == len(s2)
    return sum(c1 != c2 for c1, c2 in zip(s1, s2))

In [4]:
# transforms an int into a string binary representation with n bits
get_bin = lambda x, n=DEFAULT_BITS: format(x, 'b').zfill(n)

In [6]:
# Computes the hamming distances between the predictions (row indices) and reality (columns indices)
rows = range(8)
cols = [0, 1, 3, 7]
df = pd.DataFrame(index = rows, columns = cols)
for i in range(8):
    for j in cols:
        df[j][i] = hamming2(get_bin(i), get_bin(j))
df

,0,1,3,7
0,0,1,2,3
1,1,0,1,2
2,1,2,1,2
3,2,1,0,1
4,1,2,3,2
5,2,1,2,1
6,2,3,2,1
7,3,2,1,0


In [7]:
# computes the probability of each prediction wrt reality
prob_df = P_INCORRECT**df * P_CORRECT**(3 - df)
prob_df

,0,1,3,7
0,0.512,0.128,0.032,0.008
1,0.128,0.512,0.128,0.032
2,0.128,0.032,0.128,0.032
3,0.032,0.128,0.512,0.128
4,0.128,0.032,0.008,0.032
5,0.032,0.128,0.032,0.128
6,0.032,0.008,0.032,0.128
7,0.008,0.032,0.128,0.512


In [8]:
def swap(x, val):
    if(x < val):
        return -(1-x)
    else:
        return x

In [9]:
def ev(x, val):
    if(x < val):
        return 'sell'
    else:
        return 'buy'

In [10]:
# Computes the amount we should invest depending on the prediction, a negative amount 
# indicates we should short
summed_df = pd.DataFrame(index = range(8), columns = ['sell', 'buy'])
summed_df['sell'] = prob_df[0]*STRONG + prob_df[1]*NORMAL
summed_df['buy'] = prob_df[3]*NORMAL + prob_df[7]*STRONG
summed_df['sum'] = summed_df['buy'] + summed_df['sell']
#summed_df['coeff'] = max(summed_df['buy']/summed_df['sum'], summed_df['sell']/summed_df['sum'])
summed_df['ratio'] = summed_df['buy']/summed_df['sum']

decision_df = summed_df.copy().drop(['sell', 'buy', 'sum'],axis=1)
decision_df['choice'] = summed_df['ratio'].apply(lambda x: ev(x, 0.5))
decision_df['ratio'] = summed_df['ratio'].apply(lambda x: swap(x, 0.5))
max_ratio = max(decision_df['ratio'])
decision_df['ratio'] = decision_df['ratio']/max_ratio
decision_df['amount'] = MAX_INVESTMENT*decision_df['ratio']
decision_df

,ratio,choice,amount
0,-1.000000,sell,-100.000000
1,-0.845361,sell,-84.536082
2,-0.554054,sell,-55.405405
3,0.845361,buy,84.536082
4,-0.845361,sell,-84.536082
5,0.554054,buy,55.405405
6,0.845361,buy,84.536082
7,1.000000,buy,100.000000


In [15]:
# Takes the reality realisation as input and computes the prediction by generating random probability
# to simulate the probability that an oracle is correct with probability P_CORRECT
# OUTPUTS a value between 0 and 7 (included)
def predict(real):
    x1 = int(real/4)
    x2 = int((real%4)/2)
    x3 = real%2
    #print("{}{}{}".format(x1,x2,x3))
    pred = 0
    r1 = rd.random() 
    r2 = rd.random() 
    r3 = rd.random() 
    
    if(r1 < P_CORRECT):
        pred = pred + 4*x1
    else:
        pred = pred + 4*((x1+1)%2)
    if(r2 < P_CORRECT):
        pred = pred + 2*x1
    else:
        pred = pred + 2*((x2+1)%2)
    if(r3 < P_CORRECT):
        pred = pred + x1
    else:
        pred = pred + ((x3+1)%2)
     
    return pred

In [17]:
# Gains contains respective values for strong sell, sell, buy, strong buy
# the sell values are negative this way if we predict buy but reality was sell, we will invest positive
# and multiply by negative, which will lead to a loss
gains = [- STRONG, - NORMAL, NORMAL, STRONG]
reals = [0, 1, 3,7]
gains

[-1.2, -1.02, 1.02, 1.2]

In [387]:
# compute the profit given the bet amount, the prediction, the realisation and the decision directions
def gainzzz(amount, prediction, reality, df):
    investment = amount*df['ratio'][prediction]
    coeff = gains[reals.index(reality)]
    tmp = investment * coeff
    print(coeff)
    if(investment*tmp > 0): 
        return tmp - investment
    else: 
        return tmp + investment

In [394]:
gainzzz(100, 3, 3, decision_df)

1.02


1.6907216494845443

In [255]:
# generates a random reality event, computes a prediction given the oracles probability and returns
# the profit
def simulation(amount):
    reality = reals[rd.randint(0, 4)]
    prediction = predict(reality)
    return gainzzz(amount, prediction, reality, decision_df)

In [208]:
simulation(100)

20.0

In [375]:
money = 100

stats = []
for i in range(30):
    
    #print("total at iteration: {} = {}".format(i, money))
    tmp = simulation(money)
    stats.append(tmp/money)
    money += simulation(money)
money

280.08599189427457